# *MEEB* Run Script Tutorial - Probabilistic Version

Notebook tutorial for running the Mesoscale Explicit Ecogeomorphic Barrier model (*MEEB*) v1.1 probabilistically.

Before exploring this notebook, you are encouraged to start with the notebook `run_MEEB_Deterministic.ipynb` for a tutorial on running MEEB deterministically.

For general model information and installation instructions, see the `README` in the main project directory.

To run *MEEB* probabilistically outside of this notebook, execute the `run_MEEB_Probabilistic.py` script located in the `/Tools` folder, upon which the code in this tutorial is based. The run script should be executed from the main directory to access required inputs.

Last updated: 26 June 2025

## Probabilistic Approach
Probabilistic projections in *MEEB* account for both the uncertainty in future forcing conditions (e.g., sea-level rise, storminess) and the inherent randomness of natural phenomena. We refer to these as external and intrinsic stochastic components of the system, respectfully.

External stochasticity is incorporated by running the model across discrete probability distributions of external drivers. In other words, the model simulates across a set of values for a particular external forcing variable, each with a specific probability of occurrence that collectively sum to 1. Multiple external stochstic elements can be considered together by determining the joint probability of occurence for all possible parameter value combinations.

Intrinsic stochasticity is incorporated with a Monte Carlo method that runs a batch of duplicate simulations for each bin of the external forcing probability distribution.

## Variables and Initializations

To run *MEEB* probabilistically, first import model requirements. This includes model functions stored in the `routines_meeb.py` file and the class `MEEB` from the main `meeb.py` file. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time
import scipy
from tqdm import tqdm
from matplotlib import colors
from joblib import Parallel, delayed
import os

os.chdir(os.path.pardir)  # Set the working directory to the main MEEB folder (the parent directory of the Notebooks subdirectory)

import routines_meeb as routine 
from meeb import MEEB

Check that the current working directory is the main MEEB directory (not its Notebooks subdirectory where the notebook is stored).

In [ ]:
print(os.getcwd())

### External Stochastic Elements
Now we will define the External Stochastic Elements (ExSE). The code is set up to allow up two different elements (`ExSE_A` and `ExSE_B`; e.g., RSLR and storm intensity), but we will only use one in this tutorial: the relative sea-level rise rate (RSLR). Therefore `ExSE_B_bins` is set to zero (no change) and its probability `ExSE_B_prob` to 1 (i.e., 100% probability of no change in storm intensity). In this example, we define 3 possible RSLR rates in `ExSE_A_bins`, and their probabilities of occurance `ExSE_A_prob` sum to 1.

In [ ]:
# EXTERNAL STOCHASTIC ELEMENTS (ExSE)
# RSLR
ExSE_A_bins = [0.0068, 0.0096, 0.0124]  # [m/yr] Bins of future RSLR rates up to 2050
ExSE_A_prob = [0.26, 0.55, 0.19]  # Probability of future RSLR bins (must sum to 1.0)

# Mean Storm Intensity
ExSE_B_bins = [(0, 0)]  # [%] Bins of percent shift in mean storm intensity relative to baseline, in tuples (start, end)
ExSE_B_prob = [1]  # Probability of future storm intensity bins (must sum to 1.0)

Next, we will define values, labels, and colors for classifying each cell of the model domain each time step. Here we use 2 classifications schemes: elevation change and frequency of inundation. Elevation change bins each cell based on the difference in elevation of the cell at each timestep relative to its elevation at the simulation start. The inundation classification reflects the number of times each cell is inundated by high-water events, cumulative over time.

In [ ]:
# CLASSIFICATION SCHEME SPECIFICATIONS
elev_classification_label = 'Elevation Change [m]'  # Axes labels on figures
elev_class_edges = [-np.inf, -0.5, -0.1, 0.1, 0.5, np.inf]  # [m] Elevation change
elev_class_labels = ['< -0.5', '-0.5 - -0.1', '-0.1 - 0.1', '0.1 - 0.5', '> 0.5']
elev_class_cmap = colors.ListedColormap(['#ca0020', '#f4a582', '#f7f7f7', '#92c5de', '#0571b0'])  # Red, light red, white, light blue, blue
elev_num_classes = len(elev_class_edges) - 1 # Count number of classes in classification schemes
cmap_conf = plt.get_cmap('BuPu', 4)  # Colormap for confidence in projection, 4 discrete colors

### Simulation Specifications
Next, we will define important simulation input parameters. Any input parameters not defined here (there are many more than given below!) will follow their *default* values/specifications defined in the `meeb.py` file.

For this tutorial, we will run a 10-yr-long probabilistic forecast for a 250 m section of North Core Banks, NC, USA, beginning October 2018 and ending October 2028. You can experiment by change the parameters values below. For example changing the alongshore domain boundary min and max will change the location on North Core Banks and/or size of the model domain. The initial elevation and vegetation file used in this tutorial covers only a 5 km portion of North Core Banks; the init file that covers the entirety of North Core Banks is too large for storing in this repository.

This example is set up to use 3 duplicate simulations for each RSLR bin; this is an order of magnitude lower than recommended, but will run much faster for the sake of this tutorial. We will also use a cellsize of 2 m.

In [ ]:
# INITIAL ELEVATION AND VEGETATION FILE
start = "Init_NCB-NewDrum-Ocracoke_2018_PostFlorence_18400-23400_2m.npy"  # Initial elevation and vegetation file, stored in the Input folder
startdate = '20181007'  # [yyyymmdd] Date at which to begin the simulation; should be date of initial elevation/vegetation capture

# INITIAL PARAMETERS

sim_duration = 10  # [yr] Note: For probabilistic projections, use a duration that is divisible by the save_frequency
save_frequency = 1  # [yr] Time step for probability calculations
MHW_init = 0.39  # [m NAVD88] Initial mean high water

duplicates = 3  # Number of duplicate simulations for each ExSE bin combination to account for intrinsic stochasticity

# Define Horizontal and Vertical References of Domain
alongshore_domain_boundary_min=250  # [cellsize] Alongshore minimum coordinate
alongshore_domain_boundary_max=375  # [cellsize] Alongshore maximum coordinate
crossshore_domain_boundary_min=50  # [cellsize] Cross-shore minimum coordinate
crossshore_domain_boundary_max=425  # [cellsize] Cross-shore maximum coordinate
cellsize = 2  # [m] Horizontal cell dimensions

name = '16Sep24, 250-375, 2018-2028, n=3'  # Name of simulation suite

To reduce memory costs, we will load the initial elevation file once and pass it to the simulations, rather than have each individual simulation load it on their own.

In [ ]:
# Load Initial Domains
Init = np.load("Input/" + start)
topo_start = Init[0, alongshore_domain_boundary_min: alongshore_domain_boundary_max, crossshore_domain_boundary_min: crossshore_domain_boundary_max]
spec1_start = Init[1, alongshore_domain_boundary_min: alongshore_domain_boundary_max, crossshore_domain_boundary_min: crossshore_domain_boundary_max]
spec2_start = Init[2, alongshore_domain_boundary_min: alongshore_domain_boundary_max, crossshore_domain_boundary_min: crossshore_domain_boundary_max]
longshore, crossshore = topo_start.shape

# Count the total number times data will be saved throughout each simulation
num_saves = int(np.floor(sim_duration/save_frequency)) + 1

## Define Probabilistic Functions

The following functions run multiple duplicate simulations across each external stochastic element bin, and return arrays that classify each cell in the model domain through time. The intricacies of these functions are outside the scope of this tutorial.

The following functions 1) run an individual simulation and classify the domain for each timestep, and 2) run a batch of duplicate simulations for each RSLR bin in parallel and calculate the final joint probability across multiple RSLR bins and batch simulations. This last function, `class_probability()` is called to run the probabilistic projection.

In [ ]:
def run_individual_sim(rslr, shift_mean_storm_intensity):
    """Runs uniqe individual MEEB simulation."""

    # Create an instance of the MEEB class
    meeb = MEEB(
        name=name,
        seeded_random_numbers=False,  # This must be false to produce true randomness of natural processes
        simulation_start_date=startdate,
        simulation_time_yr=sim_duration,
        alongshore_domain_boundary_min=alongshore_domain_boundary_min,
        alongshore_domain_boundary_max=alongshore_domain_boundary_max,
        crossshore_domain_boundary_min=crossshore_domain_boundary_min,
        crossshore_domain_boundary_max=crossshore_domain_boundary_max,
        cellsize=cellsize,
        RSLR=rslr,
        shift_mean_storm_intensity_start=shift_mean_storm_intensity[0],
        shift_mean_storm_intensity_end=shift_mean_storm_intensity[1],
        MHW=MHW_init,
        save_frequency=save_frequency,
        init_by_file=False,  # This inputs elevation and vegetation directly from numpy array
        init_elev_array=topo_start,  # Numpy init elevation array
        init_spec1_array=spec1_start,  # Numpy init veg species 1 density array
        init_spec2_array=spec2_start,  # Numpy init veg species 2 density array
        saltation_length=2,  # [cells] Hop length for saltating slabs of sand
        saltation_length_rand_deviation=1,  # [cells] Deviation around saltation_length for random uniform distribution of saltation lengths
        overwash_substeps=25,
    )

    # Loop through time
    for time_step in range(int(meeb.iterations)):
        # Run time step
        meeb.update(time_step)

    classification = []
    # Create classified maps
    classification.append(classify_topo_change(meeb.topo_TS.shape[2], meeb.topo_TS))
    classification.append(classify_overwash_frequency(meeb.topo_TS.shape[2], meeb.storm_inundation_TS, meeb.topo_TS, meeb.MHW_TS))

    
    return classification


def class_probability():
    """Runs a batch of duplicate simulations, for a range of scenarios for external forcing, to find the joint classification probability
    from stochastic processes intrinsic to the system and external stochastic drivers."""

    # Create array of simulations of all parameter combinations and duplicates
    sims = np.zeros([2, len(ExSE_A_bins) * len(ExSE_B_bins)], dtype=np.int16)
    col = 0
    for a in reversed(range(len(ExSE_A_bins))):  # Run likely longest simulations (i.e., largest RSLR and storm intensity) first
        for b in reversed(range(len(ExSE_B_bins))):
            sims[0, col] = a
            sims[1, col] = b
            col += 1

    sims = np.repeat(sims, duplicates, axis=1)
    sims = sims.astype(int)
    num_sims = np.arange(sims.shape[1])

    # Run through simulations
    # This tutorial DOES NOT RUN THE SIMULATIONS IN PARALLEL, which makes it easier to understand but very slow
    # See /Tools/run_MEEB_Probabilistic.py for running in parallel using Joblib
    class_duplicates = []
    for i in tqdm(num_sims):
        out = run_individual_sim(ExSE_A_bins[sims[0, i]], ExSE_B_bins[sims[1, i]])
        class_duplicates.append(out)
    
    # ============================================================================================================

    joint_probabilities = []

    # Elevation
    joint_prob_elev = np.zeros([elev_num_classes, num_saves, longshore, crossshore], dtype=np.float16)

    # Loop through each external scenario
    for scenario in range(len(ExSE_A_bins) * len(ExSE_B_bins)):
        sim_num_start = scenario * duplicates
        sim_num_stop = sim_num_start + duplicates

        exse_a = sims[0, sim_num_start]
        exse_b = sims[1, sim_num_start]
        scenario_prob = ExSE_A_prob[exse_a] * ExSE_B_prob[exse_b]

        for c in range(elev_num_classes):
            for ts in range(num_saves):
                class_sum_ts = np.zeros([longshore, crossshore], dtype=np.float16)
                for n in range(sim_num_start, sim_num_stop):
                    class_sum_ts += (class_duplicates[n][0][ts, :, :] == c)
                joint_prob_elev[c, ts, :, :] += (class_sum_ts / duplicates) * scenario_prob

    joint_probabilities.append(joint_prob_elev)

    # Overwash Inundation Count
    joint_prob_ow = np.zeros([num_saves, longshore, crossshore], dtype=np.float16)

    # Loop through each external scenario
    for scenario in range(len(ExSE_A_bins) * len(ExSE_B_bins)):
        sim_num_start = scenario * duplicates
        sim_num_stop = sim_num_start + duplicates

        exse_a = sims[0, sim_num_start]
        exse_b = sims[1, sim_num_start]
        scenario_prob = ExSE_A_prob[exse_a] * ExSE_B_prob[exse_b]

        for ts in range(num_saves):
            class_sum_ts = np.zeros([longshore, crossshore], dtype=np.float16)
            for n in range(sim_num_start, sim_num_stop):
                class_sum_ts += class_duplicates[n][1][ts, :, :]
            joint_prob_ow[ts, :, :] += (class_sum_ts / duplicates) * scenario_prob

    joint_probabilities.append(joint_prob_ow)

    return joint_probabilities

These next two functions classify the model domain at a particular timestep according to 1) the change in elevation relative to simulation start, and 2) the cumulative number of times each cell is inundated by a high-water event.

In [ ]:

def classify_topo_change(TS, topo_TS):
    """Classify according to range of elevation change."""

    topo_change_bin = np.zeros([num_saves, longshore, crossshore], dtype=np.int8)

    for b in range(len(elev_class_edges) - 1):
        lower = elev_class_edges[b]
        upper = elev_class_edges[b + 1]

        for ts in range(TS):
            bin_change = np.logical_and(
                lower < (topo_TS[:, :, ts] - topo_TS[:, :, 0]) * (topo_TS[:, :, -1] > MHW_init),
                (topo_TS[:, :, ts] - topo_TS[:, :, 0]) * (topo_TS[:, :, -1] > MHW_init) <= upper
            ).astype(np.int8)
            topo_change_bin[ts, :, :] += bin_change * b

    return topo_change_bin


def classify_overwash_frequency(TS, inundation_TS, topo, mhw_ts):
    """Classify according to number of times inundated from storm overwash."""

    overwash = np.zeros([num_saves, longshore, crossshore], dtype=np.int16)

    for ts in range(TS):
        MHW = mhw_ts[ts]
        storm_inun = inundation_TS[:, :, ts].astype(np.int16)
        storm_inun[topo[:, :, ts] < MHW] = 0
        if ts == 0:
            overwash[ts, :, :] += storm_inun
        else:
            overwash[ts, :, :] += storm_inun + overwash[ts - 1, :, :]  # Cumulative

    return overwash

## Run the Probabilistic Projection

Finally, use the `class_probability()` function to run the probabilistic projection. This may take a while (3 - 8 minutes) given that you are running a batch of `3 * 3 = 9` total simulations.

The function returns the list `elev_class_probabilities`, which can be unwrapped to derive:

1. `elev_class_probabilities`, an array of the *probability of each elevation class across space and time* in the following format: `[class, save_timestep, crossshore, alongshore]`

2. `inundation_class_probabilities`, an array of *cumulative inundation count across space and time* in the following format: `[save_timestep, crossshore, alongshore]`

In [ ]:
joint_class_probabilities = class_probability()

elevation_class_probabilities = joint_class_probabilities[0]
overwash_class_probabilities = joint_class_probabilities[1]

## Explore Model Results

Once the model finishes, we can plot results from the class probability arrays.

### Most Likely Change in Elevation
The following function plots the most likely class across the model domain at a specified timestep (`it`).

In [ ]:
def plot_most_probable_class(class_probabilities, class_cmap, class_labels, it, orientation='vertical'):
    """Plots the most probable class across the domain at a particular time step, with separate panel indicating confidence
    in most likely class prediction. Note: this returns the first max occurance, i.e. if multiple bins are tied for the 
    maximum probability of occuring, the first one will be plotted as the most likely.

    Parameters
    ----------
    class_probabilities : ndarray
        Probabilities of each class over space and time.
    class_cmap
        Discrete colormap for plotting classes.
    class_labels : list
        List of class names.
    it : int
        Iteration to draw probabilities from.
    orientation : str
        ['vertical' or 'horizontal'] Orientation to plot domain: vertical will plot ocean along left edge of domain, 'horizontal' along bottom.
    """

    num_classes = class_probabilities.shape[0]
    mmax_idx = np.argmax(class_probabilities[:, it, :, :], axis=0)  # Find most likely class
    confidence = np.max(class_probabilities[:, it, :, :], axis=0)  # Find confidence, i.e. probability of most likely class
    min_confidence = 1 / num_classes

    # Determine whether to plot barrier horizontally or vertically
    if orientation == 'vertical':
        Fig = plt.figure(figsize=(8, 10))
        ax1 = Fig.add_subplot(121)
        ax2 = Fig.add_subplot(122)
    elif orientation == 'horizontal':
        mmax_idx = np.rot90(mmax_idx, k=1)
        confidence = np.rot90(confidence, k=1)
        Fig = plt.figure(figsize=(14, 10))
        ax1 = Fig.add_subplot(211)
        ax2 = Fig.add_subplot(212)
    else:
        raise ValueError("Orientation invalid, must use 'vertical' or 'horizontal'")

    cax1 = ax1.matshow(mmax_idx, cmap=class_cmap, vmin=0, vmax=num_classes - 1)  # Plot most likely class
    tic = np.linspace(start=((num_classes - 1) / num_classes) / 2, stop=num_classes - 1 - ((num_classes - 1) / num_classes) / 2, num=num_classes)
    mcbar = Fig.colorbar(cax1, ticks=tic)
    mcbar.ax.set_yticklabels(class_labels) # Label colorbar with class names
    plt.xlabel('Alongshore Distance [m]')
    plt.ylabel('Cross-Shore Distance [m]')

    cax2 = ax2.matshow(confidence, cmap=cmap_conf, vmin=min_confidence, vmax=1)  # Plot confidence in most likely class
    Fig.colorbar(cax2)
    plt.xlabel('Alongshore Distance [m]')
    plt.ylabel('Cross-Shore Distance [m]')

    plt.tight_layout()

Now plot the most likely ***change in elevation*** between the initial timestep and the last (i.e., `it=-1`):

In [ ]:
plot_most_probable_class(elevation_class_probabilities, elev_class_cmap, elev_class_labels, it=-1, orientation='horizontal')

### Frequency of Inundation
As another example, we can plot the cumulative number of times each cell has been inundated in a high-water event at a specific model iteration (`it`) using the following function.

In [ ]:
def plot_class_frequency(class_probabilities, it, class_label, orientation='vertical'):
    """Plots the frequency of a class (e.g., overwash inundation) across the domain at a particular time step.

    Parameters
    ----------
    class_probabilities : ndarray
        Probabilities of a class over space and time.
    it : int
        Iteration to draw probabilities from.
    class_label : str
        Name/description of class for labeling colorbar.
    orientation : str
        ['vertical' or 'horizontal'] Orientation to plot domain: vertical will plot ocean along left edge of domain, 'horizontal' along bottom.
    """

    inun_prob = class_probabilities[it, :, :]

    if orientation == 'vertical':
        Fig = plt.figure(figsize=(8, 10))
        ax1 = Fig.add_subplot(111)
    elif orientation == 'horizontal':
        inun_prob = np.rot90(inun_prob, k=1)
        Fig = plt.figure(figsize=(14, 10))
        ax1 = Fig.add_subplot(111)
    else:
        raise ValueError("plot_most_probable_class: orientation invalid, must use 'vertical' or 'horizontal'")

    cmap_class_freq = plt.get_cmap('inferno', int(np.max(inun_prob)))

    print(int(np.max(inun_prob)))

    im_ratio = inun_prob.shape[0] / inun_prob.shape[1]
    cax1 = ax1.matshow(inun_prob, cmap=cmap_class_freq, norm=colors.LogNorm())
    cb_label = 'Number of ' + class_label
    Fig.colorbar(cax1, label=cb_label, fraction=0.046 * im_ratio)
    plt.xlabel('Meters Alongshore')
    plt.ylabel('Meters Cross-Shore')

    plt.tight_layout()

Now plot the frequency of ***inundation*** at the end of the forecast (`it=-1`):

In [ ]:
plot_class_frequency(overwash_class_probabilities, it=-1, class_label='Overwash Events', orientation='horizontal')

## Want to Learn More?

Hopefully you now have a good idea how to run a probabilistic *MEEB* simulation. You can explore the effects different parameter values or settings by altering the parameters values/settings in the Simulation Specifications (box 6) or the instantiation of the `MEEB` class (box 8). Try not to make the simulations too long or large for this tutorial, particularly since individual simulations are not run in parallel in this Notebook.

`MEEB` can also be run to generate ***deterministic*** projections of future change. For a tutorial on running *MEEB* deterministically, see the `run_MEEB_Deterministic.ipynb` notebook located in the `Notebooks` folder.

MEEB is described in detail in the paper: *Reeves, I. R. B., Ashton, A. D., Lentz, E. L., Sherwood, C. R., Passeri, D. L., & Zeigler., S. L. (in review). Projecting management-relevant change of undeveloped coastal barriers with the Mesoscale Explicit Ecogeomorphic Barrier model (MEEB) v1.0: Geoscientific Model Development Discussions (preprint), https://doi.org/10.5194%2Fgmd-2024-232* 